A.S. Lundervold, v. 26.08.22

In [ ]:
# This is a quick check of whether the notebook is currently running on Google Colaboratory
# or on Kaggle, as that makes some difference for the code below.
# We'll do this in every notebook of the course.
try:
    import colab
    colab=True
except:
    colab=False

import os
kaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Setup" data-toc-modified-id="Setup-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Setup</a></span></li><li><span><a href="#The-data:-The-MNIST-data-set" data-toc-modified-id="The-data:-The-MNIST-data-set-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>The data: The MNIST data set</a></span></li><li><span><a href="#Get-the-data" data-toc-modified-id="Get-the-data-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Get the data</a></span></li><li><span><a href="#Creating-training-and-test-sets" data-toc-modified-id="Creating-training-and-test-sets-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Creating training and test sets</a></span></li><li><span><a href="#Explore-the-data" data-toc-modified-id="Explore-the-data-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Explore the data</a></span></li><li><span><a href="#Multiclass-classification-model" data-toc-modified-id="Multiclass-classification-model-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Multiclass classification model</a></span><ul class="toc-item"><li><span><a href="#Evaluation" data-toc-modified-id="Evaluation-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Evaluation</a></span><ul class="toc-item"><li><span><a href="#Cross-validation" data-toc-modified-id="Cross-validation-7.1.1"><span class="toc-item-num">7.1.1&nbsp;&nbsp;</span>Cross-validation</a></span></li><li><span><a href="#Accuracy-on-the-test-set" data-toc-modified-id="Accuracy-on-the-test-set-7.1.2"><span class="toc-item-num">7.1.2&nbsp;&nbsp;</span>Accuracy on the test set</a></span></li><li><span><a href="#Confusion-matrix" data-toc-modified-id="Confusion-matrix-7.1.3"><span class="toc-item-num">7.1.3&nbsp;&nbsp;</span>Confusion matrix</a></span></li></ul></li></ul></li><li><span><a href="#Error-analysis" data-toc-modified-id="Error-analysis-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Error analysis</a></span><ul class="toc-item"><li><span><a href="#Analyzing-individual-errors" data-toc-modified-id="Analyzing-individual-errors-8.1"><span class="toc-item-num">8.1&nbsp;&nbsp;</span>Analyzing individual errors</a></span></li></ul></li><li><span><a href="#End" data-toc-modified-id="End-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>End</a></span></li></ul></div>

The notebook is partly based on the textbook's Chapter 3. You should also have a look at Geron's notebook: https://github.com/ageron/handson-ml2/blob/master/03_classification.ipynb.

# Introduction

This notebook continues the story in `DAT158-1.4-Binary_classification.ipynb` by looking at the evaluation of **multiclass classifiers** through an example. 

<img src="https://github.com/alu042/DAT158-2022/raw/main/notebooks/assets/MnistExamples.png">

# Setup

In [ ]:
# To display plots directly in the notebook:
%matplotlib inline

We import our standard framework:

In [ ]:
# For this notebook, we need to make sure we're using an updated 
# version of scikit-learn if using Colab:
if colab:
    !pip install scikit-learn --upgrade

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib
import sklearn

In [ ]:
# Set the directory in which to store data
NB_DIR = Path.cwd()       # Set NB_DIR to be the current working directory
DATA = NB_DIR/'data'      # The data dir is the subdirectory 'data' under NB_DIR

DATA.mkdir(exist_ok=True) # Create the data directory

# The data: The MNIST data set

We'll base our discussion on a famous benchmark dataset: **MNIST**

<img src="https://github.com/alu042/DAT158-2022/raw/main/notebooks/assets/MnistExamples.png">

MNIST consists of 70.000 examples of handwritten digits. It has been called "the machine learning equivalent of fruit flies": it's simple and well-studied but not too simple. Have a look at https://en.wikipedia.org/wiki/MNIST_database and http://yann.lecun.com/exdb/mnist/ for more details.

Our goal is to construct a system that can take an image from MNIST as input and produce the correct digit 0, ..., 9 as output. This is a **multiclass system** as there are more than two possible outcomes (in contrast to our previous example of True or False for diabetes).

<centering>
<img src="https://github.com/alu042/DAT158-2022/raw/main/notebooks/assets/MNIST-goal.png">
</centering>

Correctly classifying handwritten digits is a difficult problem.. Can you come up with features that characterize all the number 2's but none of the other digits? How can you program rules that detect only 2's?

<centering>
<img src="https://github.com/alu042/DAT158-2022/raw/main/notebooks/assets/mnist-difficult.png">
</centering>

<span style="font-size:smaller">Image from G. Hinton's Coursera course [Neural Networks for Machine Learning](https://www.cs.toronto.edu/~hinton/coursera_lectures.html), now [discontinued](https://twitter.com/geoffreyhinton/status/1085325734044991489) </span>

# Get the data

[OpenML](https://www.openml.org/) is a convenient source (among many others) of machine learning data, containing MNIST among many other standard data sets. Scikit-learn has a method we can use to fetch data from OpenML:

In [ ]:
from sklearn.datasets import fetch_openml

In [ ]:
mnist = fetch_openml('mnist_784', version=1, data_home='./data', return_X_y=True)

Each of the 70.000 images are of size 28*28 = 784, 

In [ ]:
mnist[0].head()

There are 70.000 labels, one for each image (i.e. each row of the data frame):

In [ ]:
mnist[1].shape

We store the features in X and the target labels in y, as usual:

In [ ]:
X = mnist[0]
y = mnist[1]

# Creating training and test sets

As we've discussed, after collecting the data to be studied, the first step is to set aside a test set. 

> We're not interested in how well our models perform on the training set; we're really after how well they generalize to unseen data.

<img width=50% src="https://github.com/alu042/DAT158-2022/raw/main/notebooks/assets/testsplit.png"> 

We'll use the first 60.000 images as our training data and the last 10.000 as test (this is the standard split for MNIST):

In [ ]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

# Explore the data

As always, we should take a look at the training data. Having a feeling for the data's variation and quality and a sense of the task's difficulty is crucial for constructing machine learning models. When dealing with images, it's, of course, convenient to plot them.

In [ ]:
some_digit = np.array(X_train.iloc[34500])

The images are vectors of length 784:

In [ ]:
some_digit.shape

In [ ]:
28*28

To plot them we reshape to 28*28:

In [ ]:
some_digit_image = some_digit.reshape(28,28)

Here's a small section of the image:

In [ ]:
some_digit_image[15:20,15:20]

The numbers represent grayscale values.

In [ ]:
# Plot the image using a grayscale colormap
plt.imshow(some_digit_image, cmap=matplotlib.cm.binary)
plt.axis('off')
plt.show()

Let's make a small convenience function to plot MNIST images:

In [ ]:
def plot_digit(data):
    data = np.array(data)
    image = data.reshape(28,28)
    plt.imshow(image, cmap=matplotlib.cm.binary)
    plt.axis("off")

In [ ]:
plot_digit(X_train.iloc[1234])

...and plot a random selection:

In [ ]:
import random

In [ ]:
nb=10
to_plot = random.choices(X_train.values, k=nb)

In [ ]:
f = plt.figure(figsize=(14,14))
for i in range(nb):
    plt.subplot(1,nb,i+1)
    plot_digit(to_plot[i])
plt.show()

> **Your turn!** Make a function that plots a random selection of images from a specified class. For example `plot_images(image_class='8', nb=25)` should plot 25 random 8's from the training data:

<img src="https://github.com/alu042/DAT158-2022/raw/main/notebooks/assets/plot_MNIST_images.png">

You'll want to create such small throwaway helper functions all the time when you're investigating a new data set.

# Multiclass classification model

In the previous notebook, we saw some major concepts and techniques in binary classification. Let's try to predict something more challenging: all the ten classes in MNIST.

The ideas behind multiclass predictions are the same, but the difficulty for our models is increased. 

Superficially, not much changes. We can use the same scikit-learn models as before because scikit-learn takes care of adapting models for multiclass predictions—for example, `SGDClassifier` and the `RandomForestClassifier`.

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
sgd_clf = SGDClassifier(random_state=42)
rf_clf = RandomForestClassifier(random_state=42)

As we've discussed, for the `SGDClassifier` it is important to normalize the data:

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
std = StandardScaler()
X_train_std = std.fit_transform(X_train)
X_test_std = std.transform(X_test)

We can then fit the models and make predictions:

In [ ]:
sgd_clf.fit(X_train_std, y_train)

In [ ]:
# Random forests do not care about normalization, so we can use the original X_train
rf_clf.fit(X_train, y_train)

In [ ]:
some_digit = X_train.iloc[0]
plot_digit(some_digit)

Here are the predictions from our two models:

In [ ]:
sgd_clf.predict([some_digit])

In [ ]:
rf_clf.predict([some_digit])

Behind the scenes, scikit-learn trained 10 binary classifiers for us, and used them all on the `some_digit` data point. The class whose decision score was the highest was the ouput from `predict`:

In [ ]:
sgd_clf.decision_function([some_digit])

In [ ]:
# Return the position of the element with the highest value in the array. 
# That is the model's prediction.

np.argmax(sgd_clf.decision_function([some_digit]))

We can do the same with random forest classifiers by asking for the list of probabilities that the random forest assigned to each class:

In [ ]:
rf_clf.predict_proba([some_digit])

...and notice that the sixth class (corresponding to 5s) got assigned the highest probability.

In [ ]:
np.argmax(rf_clf.predict_proba([some_digit]))

## Evaluation

We can then use the same evaluation techniques that we used in the previous notebook:

In [ ]:
from sklearn.metrics import accuracy_score
#from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

In [ ]:
from sklearn.metrics import accuracy_score

### Cross-validation

We first compute predictions on all the training data using `cross_val_predict`:

In [ ]:
# Warning: These computations take some time (depending on your CPU)...
sgd_train_pred = cross_val_predict(sgd_clf, X_train_std, y_train, n_jobs=-1)
rf_train_pred = cross_val_predict(rf_clf, X_train, y_train, n_jobs=-1)

Then we can compute the accuracy on the training data (NB: note that we used `cross_val_predict` which means that the predictions are on data unseen during model fitting/training).

In [ ]:
accuracy_score(y_train, sgd_train_pred)

In [ ]:
accuracy_score(y_train, rf_train_pred)

In [ ]:
#cross_val_score(sgd_clf, X_train_std, y_train, cv=3, scoring="accuracy", n_jobs=-1)

#cross_val_score(rf_clf, X_train, y_train, cv=3, scoring="accuracy", n_jobs=-1)

### Accuracy on the test set

Based on the above cross-validation performance, we may decide to work more on our model selection steps (feel free to try out whatever you can think of!). Once that's completely done, we can evaluate the model on the test set to get our estimated generalization performance:

In [ ]:
accuracy_score(y_test, sgd_clf.predict(X_test))

In [ ]:
accuracy_score(y_test, rf_clf.predict(X_test))

### Confusion matrix


Let's compare the two classifiers' confusion matrices:

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
sgd_cm = confusion_matrix(y_train, sgd_train_pred)
rf_cm = confusion_matrix(y_train, rf_train_pred)

In [ ]:
print("SGD Confusion Matrix:")
print(sgd_cm)
print()
print("RF Confusion Matrix:")
print(rf_cm)

As we've seen, they can also be displayed in a more visually appealing way using `ConfusionMatrixDisplay`:

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
preds = [sgd_train_pred, rf_train_pred]
titles = ["SGDClassifier", "Random Forest"]
fig, axes = plt.subplots(1, 2, figsize=(14,6))
for i, ax in enumerate(axes.flat):
    ConfusionMatrixDisplay.from_predictions(y_train, preds[i], ax=ax)
    ax.set_title(titles[i])

Both confusion matrices look pretty good: most of the images land on the diagonal. However, there are errors, and there seem to be some patterns in them... So let's take a closer look.

> As a side note: these are not the kinds of models you would use for computer vision problems these days. You've heard of **"deep learning"** and the **revolution** it has caused in machine learning during the last couple of years. Where it started, and where it's made the most impact until now, is in computer vision. Using deep learning, it is straightforward to reach models that score almost perfectly on MNIST. Very few images will be misclassified (approx. 30 of the 10.000 test images). And for some of these images, we would agree with the machine's predictions if we take a look ourselves. Modern deep learning has blown away MNIST as a benchmark by essentially "solving" it. 

Another side-note: **Accuracy, precision, recall in multiclass settings**

> In  a previous exercise, I asked you to consider how to define precision and recall in a multiclass setting. Think about it again. See if you can figure out how the following numbers are computed: 

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_train, rf_train_pred, digits=3))

# Error analysis

We can zoom in on the errors in the confusion matrices by disregarding the main diagonal. But, first, we have to scale each value by the number of images in the corresponding class to properly compare error rates. Then we put zeros on the diagonal to better see the patterns elsewhere:

In [ ]:
row_sums = sgd_cm.sum(axis=1, keepdims=True)
norm_sgd_cm = sgd_cm / row_sums
np.fill_diagonal(norm_sgd_cm, 0)

In [ ]:
f, ax = plt.subplots(figsize=(12,8))
ax.matshow(norm_sgd_cm)
ax.set_xticks(range(10))
ax.set_yticks(range(10))
ax.set_xlabel("Predicted label")
ax.set_ylabel("True label")
plt.show()

In [ ]:
row_sums = rf_cm.sum(axis=1, keepdims=True)
norm_rf_cm = rf_cm / row_sums
np.fill_diagonal(norm_rf_cm, 0)

In [ ]:
f, ax = plt.subplots(figsize=(12,8))
ax.matshow(norm_rf_cm)
ax.set_xticks(range(10))
ax.set_yticks(range(10))
ax.set_xlabel("Predicted label")
ax.set_ylabel("True label")
plt.show()

We notice several interesting things:
- For the SGDClassifier, the entire column 8 is bright --> many 8s get misclassified, most frequently as 5s.
- The patterns of errors in the SGDClassifier and the Random Forest classifier is quite different. This can lead us to prefer one over the other, depending on which class confusions we care most about. 

This gives us some ideas for improving our model. Perhaps we should try to improve the 8 and 9 classifications in the random forest, or work to fix the 8/5 confusion for the SGDClassifier.

## Analyzing individual errors

Let's plot some 3s and 5s. Some 3s that the model got correct, some it confused for 5s. Some 5s the confused for 3s, and some correct 5s. 

In [ ]:
# Code taken from the book _Géron: Hands-On Machine Learning with Scikit-Learn and TensorFlow_
def plot_digits(instances, images_per_row=10, **options):
    size = 28
    images_per_row = min(len(instances), images_per_row)
    images = [instance.reshape(size,size) for instance in instances.values]
    n_rows = (len(instances) - 1) // images_per_row + 1
    row_images = []
    n_empty = n_rows * images_per_row - len(instances)
    images.append(np.zeros((size, size * n_empty)))
    for row in range(n_rows):
        rimages = images[row * images_per_row : (row + 1) * images_per_row]
        row_images.append(np.concatenate(rimages, axis=1))
    image = np.concatenate(row_images, axis=0)
    plt.imshow(image, cmap = matplotlib.cm.binary, **options)
    plt.axis("off")

In [ ]:
def individual_errors(clf, cl_a, cl_b):
    cl_a, cl_b = str(cl_a), str(cl_b)
    X_aa = X_train[(y_train == cl_a) & (sgd_train_pred == cl_a)] # Correct class a's
    X_ab = X_train[(y_train == cl_a) & (sgd_train_pred == cl_b)] # a's predicted as b's
    X_ba = X_train[(y_train == cl_b) & (sgd_train_pred == cl_a)] # b's predicted as a's
    X_bb = X_train[(y_train == cl_b) & (sgd_train_pred == cl_b)] # Correct b's
    
    plt.figure(figsize=(10,10))
    plt.subplot(221); plot_digits(X_aa[:25], images_per_row=5)
    plt.subplot(222); plot_digits(X_ab[:25], images_per_row=5)
    plt.subplot(223); plot_digits(X_ba[:25], images_per_row=5)
    plt.subplot(224); plot_digits(X_bb[:25], images_per_row=5)

In [ ]:
individual_errors(sgd_clf, 3, 5)

The SGDClassifier is a linear model. All it does is assign a weight per class to each pixel, and when it sees a new image it just sums up the wighted pixel intensities to get a score for each class. Since 3s and 5s only differ only by a few pixels, this model will easily confuse them. Same for 7s and 9s:

In [ ]:
individual_errors(sgd_clf, 7, 9)

One would expect it to perform better on for example 1s and 4s:

In [ ]:
individual_errors(sgd_clf, 1, 4)

> **Your turn!** Do a similar error analysis for the Random Forest classifier.

# End

Now that you've learned about some of the main practical machine learning ideas and techniques, you're actually already equipped to attack real-world classification problems.

However, we've treated the models as black boxes. **We haven't really discussed how machine learning models work!** It's important to know some of the hows behind the methods to be an effective machine learning practitioner. We'll get to that in Module 2 of the course.